In [ ]:
!pip install elasticsearch

In [ ]:
from elasticsearch import Elasticsearch
import pandas as pd

In [ ]:
es = Elasticsearch()


try:
  es.indices.create(index='docs')
except Exception as e:
  es.indices.delete(index='docs')
  es.indices.create(index='docs')

In [ ]:
metadata_df = pd.read_csv("../input/trec-covid-information-retrieval/CORD-19/CORD-19/metadata.csv")
topic_df = pd.read_csv("../input/trec-covid-information-retrieval/topics-rnd3.csv")

In [ ]:
for index, row in metadata_df.iterrows():
    es.index(index='docs',id=index,document={
            "cord_uid": row["cord_uid"],
            "abstract": row["abstract"] if not pd.isna(row["abstract"]) else ''
        })

In [ ]:
topic_ids = []
cord_ids = []

for index, row in topic_df.iterrows():
    res= es.search(index='docs', body={
        'query':{
            'match':{
                "abstract":row["query"]
            }
        }
    })
    for hit in res['hits']['hits']:
        cord_ids.append(hit['_source']['cord_uid'])
        topic_ids.append(row["topic-id"])

In [ ]:
submission_df = pd.DataFrame({'topic-id': topic_ids, 'cord-id': cord_ids})

In [ ]:
submission_df.to_csv("submission.csv", index=False)